code 変更箇所の記載
- 引数
- メトリック記録

local 環境でのテスト

azure machine learning training

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()

In [3]:
dataset = Dataset.get_by_name(ws, name='cifar10')

In [4]:
experiment = Experiment(workspace=ws, name='dummy-hyperdrive')

config = ScriptRunConfig(source_directory='./code/pytorch-hyperdrive',
                         script='train.py',
                         compute_target='cpu-cluster',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_mount(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])



In [5]:
env = Environment.get(ws, "pytorch-env")

In [6]:
config.run_config.environment = env

In [7]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

ps = RandomParameterSampling(
    {
        '--learning_rate': loguniform(-6, -1),
        '--momentum': loguniform(-6, -1),
    }
)

In [8]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [9]:

hyperdrive_config = HyperDriveConfig(run_config=config,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='train_loss',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [10]:
run = experiment.submit(hyperdrive_config)
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…